In [ ]:
from itertools import combinations

ORIGINAL_DATASET = ((1,2),(1,3,4),(2,5,7),(1,2,3),(2,3,5,6))
MIN_SUPPORT = 2
MIN_CONFIDENCE = 50.0

c1 = dict()

for itemset in ORIGINAL_DATASET:
    for i in itemset:
        c1[i] = c1.get(i,0) + 1

print("C1:", c1)

C1: {1: 3, 2: 4, 3: 3, 4: 1, 5: 2, 7: 1, 6: 1}


In [6]:
for item in list(c1):
    if c1[item]<MIN_SUPPORT:
        del c1[item]
print(f"C1: after Min Support Filter {MIN_SUPPORT}", c1)

C1: after Min Support Filter 2 {1: 3, 2: 4, 3: 3, 5: 2}


In [8]:

items = list(c1.keys())
support = [c1]

items

[1, 2, 3, 5]

In [11]:

for i in range(2,len(items)):
    s = dict()
    print(f"Combinations of {i} items:")
    for combo in combinations(items, i):
        print(combo)


Combinations of 2 items:
(1, 2)
(1, 3)
(1, 5)
(2, 3)
(2, 5)
(3, 5)
Combinations of 3 items:
(1, 2, 3)
(1, 2, 5)
(1, 3, 5)
(2, 3, 5)


In [ ]:
# This code implements the Apriori algorithm for frequent itemset mining

# Loop through each possible itemset size (starting from 2)
for i in range(2, len(items)):
    s = dict()  # Dictionary to store itemsets and their support counts
    
    # Generate all possible combinations of size i from the frequent items
    for combo in combinations(items, i):
        # Count how many times this combination appears in the dataset
        for itemset in ORIGINAL_DATASET:
            # Check if the current combination is a subset of the itemset
            if set(combo).issubset(itemset):
                s[combo] = s.get(combo, 0) + 1
        
        # Prune: Remove combinations that don't meet minimum support threshold
        if s.get(combo) and s[combo] < MIN_SUPPORT:
            del s[combo]
    
    # If no combinations meet the minimum support, stop the algorithm
    if not s:
        break
        
    # Add this level's frequent itemsets to our support list
    support.append(s)

# Print all discovered frequent itemsets with their support counts
print(support)


[{1: 3, 2: 4, 3: 3, 5: 2}, {(1, 2): 2, (1, 3): 2, (2, 3): 2, (2, 5): 2}]


In [ ]:
# Generate association rules from frequent itemsets
rules = dict()

# Iterate through each combination in the last level of frequent itemsets (largest size)
for combo in support[-1]:
    # For each item in the combination, create rules where this item is the consequent
    for item in combo:
        # Create antecedent by removing the current item from the combination
        # An antecedent is the "if" part (left side) of an association rule
        # For example, in a rule A→B, A is the antecedent and B is the consequent
        # Here we're creating rules by treating each item as a consequent,
        # and the remaining items in the combination as the antecedent
        c = list(combo)
        c.remove(item)
        len_c = len(c)
        
        # Format the antecedent properly (as single item or tuple)
        c = c[0] if len_c == 1 else tuple(c)
        
        # Calculate confidence for rule: item -> antecedent
        # Confidence = support(combo) / support(item)
        rule_1 = support[-1][combo]/support[0][item]*100
        
        # Calculate confidence for rule: antecedent -> item
        # Confidence = support(combo) / support(antecedent)
        rule_2 = support[-1][combo]/support[len_c-1][c]*100
        
        # Add rules that meet minimum confidence threshold to results
        if rule_1 >= MIN_CONFIDENCE: 
            rules[f"{item}->{c}"] = rule_1
        if rule_2 >= MIN_CONFIDENCE: 
            rules[f"{c}->{item}"] = rule_2

# Print all generated association rules with their confidence values
print(rules)